This notebook pulls directly from: https://github.com/kapadias/mediumposts/blob/master/nlp/published_notebooks/Evaluate%20Topic%20Models.ipynb

In [66]:
if 0 == 1:
    import nltk
    nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cfowle\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [54]:
if 0 == 1:
    !pip install gensim
    !pip install PyLDAvis
    !pip install spacy
    !python -m spacy download en_core_web_sm

  Created wheel for smart-open: filename=smart_open-2.0.0-py3-none-any.whl size=101346 sha256=ac0b8ef83106567250d16df0a4791b1f2695e10572df588e26bac61d5032c4fb
  Stored in directory: c:\users\cfowle\appdata\local\pip\cache\wheels\bb\1c\9c\412ec03f6d5ac7d41f4b965bde3fc0d1bd201da5ba3e2636de
Successfully built smart-open
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.15
    Uninstalling Cython-0.29.15:
      Successfully uninstalled Cython-0.29.15
  Attempting uninstall: docutils
    Found existing installation: docutils 0.16
    Uninstalling docutils-0.16:
      Successfully uninstalled docutils-0.16
  Created wheel for PyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97716 sha256=53f3d3161d9179bf0fe9a907e834db5d13f9ba1a2503e520a8be9b945eb87b7d
  Stored in directory: c:\users\cfowle\appdata\local\pip\cache\wheels\3b\fb\41\e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32045

  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.0-py3-none-any.whl size=12048612 sha256=e843ef06fa4c70e6f7334c5608a7d37b7f45e1102fbbfdc212a139073348e1d6
  Stored in directory: C:\Users\cfowle\AppData\Local\Temp\pip-ephem-wheel-cache-m24_lhgm\wheels\71\4a\56\e48f8ad9359a6780edd8cdd42955519b1a21d6365ad15628a2
Successfully built en-core-web-sm
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [84]:
import pandas as pd
import numpy as np

import re
import spacy
import tqdm

import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.corpora as corpora

from nltk.corpus import stopwords

from pprint import pprint

import pyLDAvis.gensim
import pickle 
import pyLDAvis

In [9]:
one_drive_path = "C:/Users/cfowle/The Estée Lauder Companies Inc/TeamAnis - General/"

In [13]:
FY20 = pd.read_excel(one_drive_path + "Data/Consumer Care/July_2019-April_2020_Merged_Feedback.xlsx")
FY19 = pd.read_csv(one_drive_path + "Data/Consumer Care/FY19.csv")
FY18 = pd.read_excel(one_drive_path + "Data/Consumer Care/FY18.xlsx")
FY17 = pd.read_excel(one_drive_path + "Data/Consumer Care/FY17.xlsx")

##combine different years
FY19 = FY19.rename(columns = {"Case Date Added": "Case Date",
                   "Mapped Brand Name": "Mapped Brand",
                   "Store Door Name": "Store Door"})
FY19["P6"] = FY19["Product Code"]

FY18 = FY18.rename(columns = {"Case Date Added": "Case Date",
                               "Mapped Brand Name": "Mapped Brand",
                               "Store Door Name": "Store Door"})
FY18["P6"] = FY18["Product Code"]

FY17 = FY17.rename(columns = {"Case Date Added": "Case Date",
                               "Mapped Brand Name": "Mapped Brand",
                               "Store Door Name": "Store Door"})
FY17["P6"] = FY17["Product Code"]

cc_codes = pd.concat([FY20, FY19, FY18, FY17], axis = 0)
cc_codes["Date Month"] = pd.to_datetime(cc_codes['Date Month'])
cc_codes = cc_codes.rename(columns = {"Date Month": "month"})
cc_codes['P6'] = ['nan' if x[0] == 'nan' else str(x[0])[:6] for x in cc_codes[["P6"]].values]

In [14]:
cc_codes.to_csv(one_drive_path + "Data/Consumer Care/merged_cc_F17_FY20.csv")

## Clean Text

In [57]:
cc_codes  = cc_codes.dropna(subset = ["Verbatim"])
cc_codes  = cc_codes.loc[cc_codes["Reportable Country"] == 'United States of America']
documents = cc_codes["Verbatim"]

In [58]:
documents = documents.map(lambda x: re.sub('[,\.!?]', '', x))
documents = documents.map(lambda x: x.lower())

In [59]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = documents.values.tolist()
data_words = list(sent_to_words(data))

In [61]:
data_words[1]

['can',
 'you',
 'help',
 'me',
 'find',
 'out',
 'this',
 'information',
 'found',
 'this',
 'product',
 'in',
 'my',
 'closet',
 'in',
 'the',
 'way',
 'back']

In [63]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [68]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [69]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [70]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [71]:
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [72]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [74]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['large', 'red', 'cystic', 'bump', 'face', 'initially', 'think', 'maybe', 'purge', 'give', 'sometime', 'keep', 'get', 'bad']


In [77]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]


In [86]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=7, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [87]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.047*"store" + 0.036*"go" + 0.029*"want" + 0.028*"buy" + 0.024*"tell" + '
  '0.024*"say" + 0.022*"return" + 0.020*"get" + 0.020*"purchase" + '
  '0.019*"product"'),
 (1,
  '0.073*"order" + 0.032*"get" + 0.030*"online" + 0.029*"receive" + '
  '0.025*"gift" + 0.020*"email" + 0.020*"place" + 0.018*"call" + 0.015*"card" '
  '+ 0.015*"want"'),
 (2,
  '0.065*"use" + 0.035*"skin" + 0.026*"face" + 0.024*"get" + 0.022*"eye" + '
  '0.022*"day" + 0.019*"product" + 0.017*"go" + 0.017*"time" + 0.015*"start"'),
 (3,
  '0.029*"open" + 0.026*"damage" + 0.025*"box" + 0.022*"send" + '
  '0.022*"product" + 0.017*"top" + 0.014*"salon" + 0.012*"replacement" + '
  '0.011*"picture" + 0.011*"break"'),
 (4,
  '0.064*"would" + 0.058*"product" + 0.026*"love" + 0.021*"know" + '
  '0.020*"like" + 0.015*"could" + 0.013*"sample" + 0.012*"want" + '
  '0.012*"company" + 0.012*"ingredient"'),
 (5,
  '0.053*"product" + 0.025*"hair" + 0.024*"use" + 0.023*"year" + '
  '0.017*"purchase" + 0.017*"bottle" + 0.015*"b

In [80]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.5707012021165434


In [81]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [82]:
grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
               corpus]

corpus_title = ['100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

  1%|▏         | 4/270 [1:38:25<108:18:19, 1465.79s/it]

KeyboardInterrupt: 

In [88]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.150048 -0.118787       1        1  20.012102
5      0.070020  0.069810       2        1  15.807926
4      0.024586  0.043639       3        1  15.266879
2      0.132043 -0.036537       4        1  14.823845
6     -0.003691  0.333769       5        1  13.913451
1      0.049273 -0.223394       6        1  12.986086
3     -0.422279 -0.068500       7        1   7.189713, topic_info=         Term          Freq         Total Category  logprob  loglift
28       look  62265.000000  62265.000000  Default  30.0000  30.0000
135     order  43954.000000  43954.000000  Default  29.0000  29.0000
64        use  61673.000000  61673.000000  Default  28.0000  28.0000
22      color  40396.000000  40396.000000  Default  27.0000  27.0000
75      store  43833.000000  43833.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
222      come   2690.496582  17067.542969   Topic7  -4.8142   0.7851
1140    state   1649.718384   2917.657715   Topic7  -5.3034   2.0623
302   package   1794.194458   5315.109375   Topic7  -5.2194   1.5465
411    pencil   1659.189697   6747.393555   Topic7  -5.2976   1.2297
14       back   1501.735229  27127.966797   Topic7  -5.3973  -0.2614

[446 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
2047      7  0.999481         accept
370       3  0.308106        account
370       6  0.691726        account
549       4  0.999267           acne
806       5  0.998834  acne_solution
...     ...       ...            ...
66        1  0.060010           year
66        2  0.604992           year
66        3  0.040688           year
66        4  0.133972           year
66        5  0.160306           year

[684 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 6, 5, 3, 7, 2, 4])